### Akin - Example Usage

In [1]:
from akin import UniMinHash, LSH
import pandas as pd

In [2]:
content = pd.DataFrame(
    {
        'text': [
            'Jupiter is primarily composed of hydrogen with a quarter of its mass being helium',
            'Jupiter moving out of the inner Solar System would have allowed the formation of inner planets.',
            'A helium atom has about four times as much mass as a hydrogen atom, so the composition changes '
            'when described as the proportion of mass contributed by different atoms.',
            'Jupiter is primarily composed of hydrogen and a quarter of its mass being helium',
            'A helium atom has about four times as much mass as a hydrogen atom and the composition changes '
            'when described as a proportion of mass contributed by different atoms.',
            'Theoretical models indicate that if Jupiter had much more mass than it does at present, it '
            'would shrink.',
            'This process causes Jupiter to shrink by about 2 cm each year.',
            'Jupiter is mostly composed of hydrogen with a quarter of its mass being helium',
            'The Great Red Spot is large enough to accommodate Earth within its boundaries.'
        ]
    }
)

In [3]:
labels = [i for i in range(1, len(content))]
labels

[1, 2, 3, 4, 5, 6, 7, 8]

**Create MinHash object:**

In [4]:
minhash = UniMinHash(n_gram=9, permutations=30, hash_bits=64, seed=3)

In [5]:
signatures = minhash.transform(content['text'])

In [6]:
content['signature'] = signatures

In [7]:
content

,text,signature
0,Jupiter is primarily composed of hydrogen with...,"(-8690990394074104221, -8039832347290236251, -..."
1,Jupiter moving out of the inner Solar System w...,"(-9163654026142366035, -8430343577706303689, -..."
2,A helium atom has about four times as much mas...,"(-9200160712590687808, -9191248622597854076, -..."
3,Jupiter is primarily composed of hydrogen and ...,"(-8690990394074104221, -8039832347290236251, -..."
4,A helium atom has about four times as much mas...,"(-9200160712590687808, -9191248622597854076, -..."
5,Theoretical models indicate that if Jupiter ha...,"(-8658016116359304645, -8281325199261098028, -..."
6,This process causes Jupiter to shrink by about...,"(-8974290714879493554, -8196896290515441543, -..."
7,Jupiter is mostly composed of hydrogen with a ...,"(-8690990394074104221, -8039832347290236251, -..."
8,The Great Red Spot is large enough to accommod...,"(-9220885796373233114, -8851563496498238350, -..."


**Create LSH object:**

In [8]:
lsh = LSH(permutations=30)
lsh.update(signatures, labels)

**Query to find near duplicates for text 1:**

In [9]:
lsh.query(1, min_jaccard=0.5)

[8, 4]

**Generate minhash signature and add new texts to LSH model:**

In [10]:
new_text = [
    'Jupiter is primarily composed of hydrogen with a quarter of its mass being helium',
    'Jupiter moving out of the inner Solar System would have allowed the formation of '
    'inner planets.'
]

In [11]:
new_labels = ['doc1', 'doc2']

In [12]:
new_signatures = minhash.transform(new_text)
lsh.update(new_signatures, labels=new_labels)

**Remove text and label from model:**

In [13]:
# Remove text and label from model.
lsh.remove([5])

In [15]:
lsh.adjacency_list(min_jaccard=0.55)

100%|██████████| 9/9 [00:00<00:00, 32236.32it/s]


{1: [8, 'doc1', 4],
 2: ['doc2'],
 3: [],
 4: [1, 'doc1'],
 6: [],
 7: [],
 8: [1, 'doc1'],
 'doc1': [8, 1, 4],
 'doc2': [2]}